# Data Cleaning and Stratified Sampling

Before I can sample the data I need to make sure the data is clean. 

List of tasks:
1. Turn NAN values to NaN
2. Remove unnecessary NA rows
3. Fill in NA cells for instructors still teaching
4. Remove duplicate names
5. Turn Biomedical Sciences into School of Medicine
6. Remove retired/passed/quit/no email instructors
7. Simplify the titles
8. Simplify college and departments
9. Simplify full name and isolate last name
10. Perform a non proportional stratified random sample of college/organization

In [2]:
# import necessary libraries
import pandas as pd
import numpy as np

In [3]:
# read in csv
df = pd.read_csv("../data/populationInstructorData.csv")

In [4]:
# before doing anything, check the head and tail to make sure the data is read in
# df.head()
df.tail()

,FullName,Email,Title,Department,College
1882,Paul Kirwan,paul.kirwan@ucr.edu,Lecturer,Academic Affairs Ops,University Extension
1883,Christine Petzar,NAN,Lecturer in Education,NAN,University Extension
1884,Lorna K Seitz,lorna.seitz@ucr.edu,Lecturer,International Programs,University Extension
1885,Nidhi Shah,nidhi.shah@ucr.edu,Lecturer,Professional Programs,University Extension
1886,Michael R Towne,michael.towne@ucr.edu,Lecturer,Custom & Community Pgms,University Extension


In [5]:
# review all column names
df.columns.values

array(['FullName', 'Email', 'Title', 'Department', 'College'],
      dtype=object)

### Turning NAN values to NaN

In [6]:
# .info doesn't understand NAN as an NA value
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1887 entries, 0 to 1886
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   FullName    1887 non-null   object
 1   Email       1887 non-null   object
 2   Title       1887 non-null   object
 3   Department  1887 non-null   object
 4   College     1887 non-null   object
dtypes: object(5)
memory usage: 73.8+ KB


In [7]:
df = df.iloc[:,:].replace("NAN",np.nan)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1887 entries, 0 to 1886
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   FullName    1887 non-null   object
 1   Email       1851 non-null   object
 2   Title       1887 non-null   object
 3   Department  1870 non-null   object
 4   College     1887 non-null   object
dtypes: object(5)
memory usage: 73.8+ KB


Initially, the NA values are labeled as NAN. Because of this, when using .info(), pandas doesn't understand and doesn't count the NAN's as NAs. This causes a problem leading us to believe that there are no null values. After replacing the NAN's with NaN, we are able to see otherwise. 

In [8]:
# look through all rows and cols for NAs
df[df.isnull().any(axis=1)].head()

,FullName,Email,Title,Department,College
0,Y Paul Aoyagi,NaN,Clinical Professor,Biomedical Sciences,Biomedical Sciences
11,Fred Z Havens,NaN,Assistant Clinical Professor,Biomedical Sciences,Biomedical Sciences
15,William Junkert Jr,NaN,Clinical Professor,Biomedical Sciences,Biomedical Sciences
18,Walter M Marcus,NaN,Clinical Professor,Biomedical Sciences,Biomedical Sciences
29,Steven Wilson MD,NaN,Assistant Clinical Professor,Biomedical Sciences,Biomedical Sciences


### Removing duplicates

In [9]:
# findin duplicated values
df[df.duplicated('FullName', keep=False)].sort_values(by='FullName').head()


,FullName,Email,Title,Department,College
1267,Adam Godzik,adam.godzik@ucr.edu,"Professor, Biomedical Sciences",Biomedical Sciences,School of Medicine
10,Adam Godzik,adam.godzik@ucr.edu,"Professor, Biomedical Sciences",Biomedical Sciences,Biomedical Sciences
27,Ameae M Walker,ameae.walker@ucr.edu,Professor of Biomedical Sciences,Biomedical Sciences,Biomedical Sciences
1635,Ameae M Walker,ameae.walker@ucr.edu,Professor of Biomedical Sciences,Biomedical Sciences,School of Medicine
31,Changcheng Zhou,changcheng.zhou@ucr.edu,"Professor, Biomedical Sciences",Biomedical Sciences,Biomedical Sciences


In [10]:
# keep the SOM duplicates and sort alphabetically by name
df = df[~df.duplicated('FullName',keep='last')].sort_values(by='FullName')

# double check for duplicates
# df[df.duplicated('FullName', keep=False)].sort_values(by='FullName')

In [11]:
# Checking for others in Biomedical sciences
df[df['College']=="Biomedical Sciences"]

,FullName,Email,Title,Department,College
32,"Natalie E. Zlebnik, Ph.D.",natalie.zlebnik@ucr.edu,Assistant Professor,Biomedical Sciences,Biomedical Sciences
22,"Scott D. Pegan, Ph.D.",scott.pegan@ucr.edu,Professor,Biomedical Sciences,Biomedical Sciences


In [12]:
df.loc[:,'College'].replace('Biomedical Sciences',"School of Medicine",inplace=True)

For the duplicated rows, the only difference between the two is the college. They are either is either Biomedical Sciences or School of Medicine. Becuase the department is already Biomedical Sciences, I decided to keep the duplicates with SOM as their college.
One other duplicate was Daniel R. Jeske. He is now the chancellor, not the provost so I chose the one without the Provost title. 

It also turned out that there were two professors that contained Biomedical Sciences as their college. I googled them and they still work for the UCR SOM, so their college was changed. 

4. [X] Remove duplicate names
5. [X] Turn Biomedical Sciences into School of Medicine


### Removing retired/deceased or instructors that have quit and filling in info for those still teaching

In [13]:
# emeritus = retired
df[df['Title'].str.lower().str.contains("emeritus|emerita|emeriti")].head()

,FullName,Email,Title,Department,College
304,Alan G Fix,alan.fix@ucr.edu,"Professor of Anthropology, Emeritus",Anthropology,"Coll of Hum, Arts & Social Sci"
222,Alan R Beals,alan.beals@ucr.edu,"Professor of Anthropology, Emeritus",Anthropology,"Coll of Hum, Arts & Social Sci"
861,Albert L Page,albert.page@ucr.edu,Professor Emeritus of Soil Science & Chemist,Environmental Sciences,College of Nat & Agr Sciences
441,Alexandra Maryanski,alexandra.maryanski@ucr.edu,Emeritus Professor,Sociology,"Coll of Hum, Arts & Social Sci"
703,Allan Dodds,allan.dodds@ucr.edu,"Professor & Plant Pathologist, Emeritus",Microbiology & Plant Pathology,College of Nat & Agr Sciences


In [14]:
# remove emeritus
df = df[~df['Title'].str.lower().str.contains("emeritus|emerita|emeriti")]

In [15]:
# list of rest of NA values. Why are they NA?
df[df.isnull().any(axis=1)].head()

,FullName,Email,Title,Department,College
1883,Christine Petzar,NaN,Lecturer in Education,NaN,University Extension
1785,Corrie Jane Neighbors,NaN,Lecturer,NaN,College of Nat & Agr Sciences
1846,Dan Hepler,NaN,Lecturer,NaN,Undergraduate Education
1863,Dio Saucedo,NaN,Lecturer,NaN,Undergraduate Education
1298,Fred Z Havens,NaN,Assistant Clinical Professor,Biomedical Sciences,School of Medicine


In [16]:
deceased = ['Fred Z Havens','William Junkert Jr']
new_practice = ['Steven Wilson MD','Uziel Sauceda','Walter M Marcus']

filters = deceased + new_practice
df[df['FullName'].str.contains("|".join(filters))]

,FullName,Email,Title,Department,College
1298,Fred Z Havens,NaN,Assistant Clinical Professor,Biomedical Sciences,School of Medicine
1651,Steven Wilson MD,NaN,Assistant Clinical Professor,Biomedical Sciences,School of Medicine
1557,Uziel Sauceda,NaN,"Assistant Clinical Professor, Health Sciences",NaN,School of Medicine
1436,Walter M Marcus,NaN,Clinical Professor,Biomedical Sciences,School of Medicine
1343,William Junkert Jr,NaN,Clinical Professor,Biomedical Sciences,School of Medicine


In [17]:
# remove known deceased/new practice
df = df[~df['FullName'].str.contains("|".join(filters))]

In [18]:
# look again at rest of NAs
df[df.isnull().any(axis=1)].head()

,FullName,Email,Title,Department,College
1883,Christine Petzar,NaN,Lecturer in Education,NaN,University Extension
1785,Corrie Jane Neighbors,NaN,Lecturer,NaN,College of Nat & Agr Sciences
1846,Dan Hepler,NaN,Lecturer,NaN,Undergraduate Education
1863,Dio Saucedo,NaN,Lecturer,NaN,Undergraduate Education
1740,Hiroko Inoue,NaN,Lecturer,NaN,"Coll of Hum, Arts & Social Sci"


In [19]:
still_teaching = {'Dan Hepler':['dan.hepler@ucr.edu','University Writing Program'], 'Jennifer Kimea Simmers':['Jennifer.simmers@ucr.edu','Sociology'], 'Kelli Cathleen King':['kelli.king@ucr.edu','Dance'], 'Ryan Rusich':['rusichr@cs.ucr.edu','Computer Science & Engineering']}

# df.loc[df['FullName'].isin(still_teaching.keys()) gives df when the col full name matches the keys of missing people
# df['Email'].isnull() finds the cols with missing values in Email col
# [, 'Email'] only shows the email column 
# dictionaries must be put in a list to subscript
df.loc[df['FullName'].isin(still_teaching.keys()) & df['Email'].isnull(), 'Email'] = [email[0] for email in list(still_teaching.values())]

df.loc[df['FullName'].isin(still_teaching.keys()) & df['Department'].isnull(), 'Department'] = [dept[1] for dept in list(still_teaching.values())]

In [20]:
# checking if info filled in
df.loc[df['FullName'].isin(still_teaching.keys())]

,FullName,Email,Title,Department,College
1846,Dan Hepler,dan.hepler@ucr.edu,Lecturer,University Writing Program,Undergraduate Education
1766,Jennifer Kimea Simmers,Jennifer.simmers@ucr.edu,Lecturer,Sociology,"Coll of Hum, Arts & Social Sci"
1744,Kelli Cathleen King,kelli.king@ucr.edu,Lecturer,Dance,"Coll of Hum, Arts & Social Sci"
1699,Ryan Rusich,rusichr@cs.ucr.edu,Lecturer,Computer Science & Engineering,Bourns College of Engineering


In [21]:
# remove rest of NA values for which no info found
df = df.loc[~df.isnull().any(axis=1)]

In [22]:
# daniel jeske doesnt teach. the chancellor 
df = df.loc[df['FullName'] != 'Daniel R. Jeske']

Turns out the profiles website has old info. It contained professors that were retired, deceased or have left. I googled each person with missing information and those below I couldnt find an email and/or information about them still working at UCR:

- Christine
- Corrie last info is 2016
- Dio
- Jennifer
- Kenneth 
- Natasha 
- Nicoletta
- Nissim
- Ramon 
- Stephanie

It seemed like most of those with missing information were lecturers.

To check for info on people I just typed their name folowed by ucr. I read their rate my professor to see if someone rated in the past year or checked their linked in if theyre at a new position

2. [X] Remove unnecessary NA rows
3. [X] Fill in NA cells for instructors still teaching
6. [X] Remove retired/passed/quit/no email instructors


### Simplify Title cells

In [23]:
df['Title'].describe()

count                             1590
unique                             285
top       Assistant Clinical Professor
freq                               364
Name: Title, dtype: object

In [24]:
# long title
df['Title'].head()

882     Professor of Plant Pathology & Microbiology
1399                   Assistant Clinical Professor
1861                                       Lecturer
535                         Professor of Psychology
182              Adjunct Professor NAVSEA Scientist
Name: Title, dtype: object

In [25]:
# abbreviated titles
df.loc[df['FullName'].isin(["Ni'Ja Whitson",'Dong-Hwan Choe'])]

,FullName,Email,Title,Department,College
670,Dong-Hwan Choe,donghwan.choe@ucr.edu,Assoc C/E Specialist and Assoc Professor,Entomology,College of Nat & Agr Sciences
591,Ni'Ja Whitson,nija.whitson@ucr.edu,Asst Professor,Dance,"Coll of Hum, Arts & Social Sci"


In [26]:
# str.replace replaces part of the string
# if just .replace() the to_replace must be the full string and will replace the full string with the value
df['Title'] = df['Title'].str.replace('Assoc|assoc', "Associate", regex=True)
df['Title'] = df['Title'].str.replace('Asst|asst', "Assistant", regex=True)


In [27]:
# double check to see that they're changed
df.loc[df['FullName'].isin(["Ni'Ja Whitson",'Dong-Hwan Choe'])]


,FullName,Email,Title,Department,College
670,Dong-Hwan Choe,donghwan.choe@ucr.edu,Associate C/E Specialist and Associate Professor,Entomology,College of Nat & Agr Sciences
591,Ni'Ja Whitson,nija.whitson@ucr.edu,Assistant Professor,Dance,"Coll of Hum, Arts & Social Sci"


In [28]:
# base titles wanted
my_dict = {"Assistant": "Assistant Professor", "Associate": "Associate Professor", "Distinguished": "Distinguished Professor", "Dean": "Dean", "Chair": "Chair", "Lecturer":"Lecturer", 'Adjunct':'Adjunct Professor'}

# df = df.drop(columns='TitleNew')

# the loop only changes for the everyone that isnt a base professor. for those that are base professors, it will put them as NaN
for key, value in my_dict.items():
    to_change = df['Title'].str.contains(key)
    df.loc[to_change,'TitleNew'] = value

df = df.fillna('Professor').drop(columns='Title')

In [29]:
df.groupby('TitleNew').nunique()

,FullName,Email,Department,College
TitleNew,,,,
Adjunct Professor,40,40,15,5
Assistant Professor,643,643,52,8
Associate Professor,297,297,51,7
Chair,39,39,27,5
Dean,7,7,5,3
Distinguished Professor,65,65,25,6
Lecturer,181,181,31,8
Professor,318,318,51,9


Many titles contain other unnecessary fillings like professor of plant path and microbiology totaling to 286 unique titles. I just want the base title like professor, associate professor, etc.

Titles that included abbreviations were made full length so they could be detected. 

Some Adjunct Professors were also Assistant/Associate professors but I put them in their own category as they only teach for a limited term. 

7. [X] Simplify the titles


### Simplifying the different College organizations and departments

In [30]:
df.groupby(['College']).nunique()

,FullName,Email,Department,TitleNew
College,,,,
Bourns College of Engineering,162,162,10,8
CE - College of Nat & Ag Sci,3,3,3,2
"Coll of Hum, Arts & Social Sci",404,404,22,7
College of Nat & Agr Sciences,262,262,15,7
Graduate School of Education,42,42,1,6
School of Business,70,70,1,8
School of Medicine,578,578,13,5
School of Public Policy,15,15,1,5
Undergraduate Education,48,48,2,2


In [31]:
temp = df.groupby('Department', as_index=False)['College'].count().sort_values(by='College')
temp

,Department,College
12,CNAS Dean's Office,1
19,Custom & Community Pgms,1
39,Material Sci/Engr Pgm,1
47,POEM Center in BCoE,1
15,Community Engagement,1
...,...,...
57,SOM Medicine,63
63,School of Business,70
58,SOM Neuroscience,72
21,Dept. of Family Medicine,125


In [32]:
# turn those with count <= 5 to other dept
temp_dept_list = temp['Department'].loc[temp['College'] <= 5].tolist()
df['Department'] = df['Department'].replace(temp_dept_list, 'Other')

# check to see if they changed
df.loc[df['Department'] == 'Other'].head()

,FullName,Email,Department,College,TitleNew
182,Aaron Wiest,aaron.wiest@ucr.edu,Other,Bourns College of Engineering,Adjunct Professor
700,Adler R. Dillman,adler.dillman@ucr.edu,Other,College of Nat & Agr Sciences,Associate Professor
195,Alexander I Putman,alexander.putman@ucr.edu,Other,CE - College of Nat & Ag Sci,Assistant Professor
1832,Anamika Basu,anamika.basu@ucr.edu,Other,Undergraduate Education,Lecturer
196,Andreas Westphal,andreas.westphal@ucr.edu,Other,CE - College of Nat & Ag Sci,Professor


In [33]:
df = df.replace('CE - College of Nat & Ag Sci', 'Coll of Hum, Arts & Social Sci')

In [34]:
df.groupby(['College']).nunique()


,FullName,Email,Department,TitleNew
College,,,,
Bourns College of Engineering,162,162,6,8
"Coll of Hum, Arts & Social Sci",407,407,22,7
College of Nat & Agr Sciences,262,262,13,7
Graduate School of Education,42,42,1,6
School of Business,70,70,1,8
School of Medicine,578,578,10,5
School of Public Policy,15,15,1,5
Undergraduate Education,48,48,2,2
University Extension,6,6,1,1


In [35]:
# splitting names by first ,|: to keep the names and remove the extra titles
df['SplitName'] = [name[0] for name in df['FullName'].str.split(',|:', 1)]
df.loc[df['FullName'] == 'Mallory A. Stuparich, MD, FACOG']


,FullName,Email,Department,College,TitleNew,SplitName
1596,"Mallory A. Stuparich, MD, FACOG",mallory.stuparich@ucr.edu,SOM OB/GYN,School of Medicine,Assistant Professor,Mallory A. Stuparich


In [36]:
# [] encapsulates a set of characters
# \ is used to escape special characters, so \( just means ( and \[ is [
# . states any character and * states zero or more occurences
# so we ask to find something that starts with ( or [ has 0 or more characters in between and ends in ) or ]

# removing nick names
df['SplitName'] = df['SplitName'].str.replace("[\(\[].*[\)\]]", '', regex=True)
df.loc[df['FullName'] == 'Katayoon (Katie) Dehesh']

,FullName,Email,Department,College,TitleNew,SplitName
696,Katayoon (Katie) Dehesh,katayoon.dehesh@ucr.edu,Botany and Plant Sciences,College of Nat & Agr Sciences,Distinguished Professor,Katayoon Dehesh


In [37]:
# person with only first name listed
df.loc[[len(x)==1 for x in df['SplitName'].str.rsplit(' ', 1)], :]


,FullName,Email,Department,College,TitleNew,SplitName
285,Latipa,latipa.x@ucr.edu,Media & Cultural Studies,"Coll of Hum, Arts & Social Sci",Associate Professor,Latipa


In [38]:
# write full name in splitname column
df.loc[df['FullName'] == 'Latipa', 'SplitName'] = 'Michelle Dizon'

# double check the change was made
df.loc[df['FullName'] == 'Latipa']

,FullName,Email,Department,College,TitleNew,SplitName
285,Latipa,latipa.x@ucr.edu,Media & Cultural Studies,"Coll of Hum, Arts & Social Sci",Associate Professor,Michelle Dizon


In [39]:
df.loc[df['SplitName'] == 'Masoumeh Ghaffari MD']

,FullName,Email,Department,College,TitleNew,SplitName
1258,Masoumeh Ghaffari MD,masoumeh.ghaffari@ucr.edu,SOM Internal Medicine,School of Medicine,Assistant Professor,Masoumeh Ghaffari MD


In [40]:
# some people still had titles in their name

from titleAbbreviations import getAbbreviations
# double check if there are still occurences with titles in name

abbr = getAbbreviations()
abbr[0:7]

['F.R.Eng.', 'AA', 'A.A.', 'AS', 'A.S.', 'BEP', 'CAP', 'Dip.Arts', 'Dip.Lang.Stud.', 'Dip.Lang', 'Dip.Soc.Sc.', 'Dip.Ed', 'Dip.Mus.', 'BA', 'B.A.', 'AB', 'A.B.', 'B.Arts', 'BSA', 'BAcy', 'B.Acy.', 'BAcc', 'B.Acc.', 'B.An.Vet.Bio.Sc.', 'B.App.Sc.', 'B.A.Sc.', 'BArch', 'B.Arch.', 'BBA', 'B.B.A.', 'BCE', 'B.C.E.', 'BComm', 'B.Comm.', 'BCom', 'B.Com.', 'B.Comm.', 'BCA', 'BDH', 'B.D.H', 'BDM', 'B.D.M.', 'BDSc', 'B.D.Sc.', 'BDS', 'B.D.S.', 'BChD', 'B.Ch.D.', 'BDent', 'B.Dent.', 'BDes', 'B.Des.', 'B.Des.Comp.', 'B.Des.Arch.', 'BEd', 'B.Ed.', 'BEng', 'B.Eng.', 'BE', 'B.E', 'BEC', 'B.E-COM.', 'BEE', 'B.E.E.', 'BFA', 'B.F.A.', 'B.Hlth.Sci', 'BIT', 'B.I.T.', 'BIGS', 'B.I.G.S.', 'LLB', 'LL.B.', 'BLAS', 'B.L.A.S.', 'BLib', 'B.Lib.', 'BLS', 'B.L.S.', 'BLit', 'B.Lit.', 'BMath', 'B.Math', 'BME', 'B.M.E.', 'B.Med.Sc.', 'MB', 'M.B.', 'B.M.', 'B.Mus.Studies', 'BN', 'B.N.', 'B.Pharm.', 'B.Ph.', 'B.Pol.', 'B.Pol.I.', 'B.P.S', 'B.P.E.S.S.', 'B.Res.Ec.', 'BS', 'B.S.', 'BSc', 'B.Sc.', 'BSDH', 'B.S.D.H', 'BS.E

['F.R.Eng.', 'AA', 'A.A.', 'AS', 'A.S.', 'BEP', 'CAP']

In [41]:
# split full name
names = df['SplitName'].str.split(" ")

# for each list 
for name in names:
    # look at each word in the name
    for word in name:
        # if one of the words equals an abbreviation, remove it
        if word in abbr:
            name.remove(word)

# join everything back
df['CleanName'] = [' '.join(x).title() for x in names]

df.loc[df['SplitName'] == 'Masoumeh Ghaffari MD']

,FullName,Email,Department,College,TitleNew,SplitName,CleanName
1258,Masoumeh Ghaffari MD,masoumeh.ghaffari@ucr.edu,SOM Internal Medicine,School of Medicine,Assistant Professor,Masoumeh Ghaffari MD,Masoumeh Ghaffari


In [42]:
# axis = 1 is col, axis = 0 is row
df = df.drop(['FullName','SplitName'], axis=1)

# getting last name only
last_names = [last_name[1] for last_name in df['CleanName'].str.rsplit(' ', 1)]

df['LastName'] = last_names
df.head()

,Email,Department,College,TitleNew,CleanName,LastName
882,a.rao@ucr.edu,Microbiology & Plant Pathology,College of Nat & Agr Sciences,Professor,A. L. N. Rao,Rao
1399,aaron.lee@ucr.edu,SOM Medicine,School of Medicine,Assistant Professor,Aaron Lee,Lee
1861,aaron.potter@ucr.edu,University Writing Program,Undergraduate Education,Lecturer,Aaron Potter,Potter
535,aaron.seitz@ucr.edu,Psychology,"Coll of Hum, Arts & Social Sci",Professor,Aaron Seitz,Seitz
182,aaron.wiest@ucr.edu,Other,Bourns College of Engineering,Adjunct Professor,Aaron Wiest,Wiest




Ran into problem where there was 1 professor that only had their first name

Another problem was that if there wasnt a comma separating the titles from the name, then they werent removed. So I needed to remove them

Create a column for last names for later so I can create a script with their name to just say Dr. Doe
However for lecturers I will just have to call them by their first and last name

8. [X] Simplify college and departments
9. [X] Simplify full name and isolate last name

### Stratified Sampling

In [43]:
# population size
N = df.shape[0]
N

1590

In [44]:
# college count
spread_sheet = df.groupby('College', as_index=False)['CleanName'].count().sort_values(by='CleanName')
spread_sheet = spread_sheet.rename(columns={'CleanName':'Nh'})

# population proportion
spread_sheet['Nh/N'] = spread_sheet['Nh'] / N

# sample size
n = 800

# stratified sample sizes
spread_sheet['nh'] = (n * spread_sheet['Nh/N']).round(0).astype(int)

# changing some of the stratified sample sizes to account for non response
spread_sheet.iloc[0:2,3] = spread_sheet.iloc[0:2,1]

# sample proportion
spread_sheet['nh/n'] = spread_sheet['nh'] / n
spread_sheet


,College,Nh,Nh/N,nh,nh/n
8,University Extension,6,0.003774,6,0.00750
6,School of Public Policy,15,0.009434,15,0.01875
3,Graduate School of Education,42,0.026415,21,0.02625
7,Undergraduate Education,48,0.030189,24,0.03000
4,School of Business,70,0.044025,35,0.04375
0,Bourns College of Engineering,162,0.101887,82,0.10250
2,College of Nat & Agr Sciences,262,0.164780,132,0.16500
1,"Coll of Hum, Arts & Social Sci",407,0.255975,205,0.25625
5,School of Medicine,578,0.363522,291,0.36375


In [45]:
np.random.seed(130)

small_df = []
for i, row in spread_sheet.iterrows():
    college = row['College']
    sample_size = row['nh']

    df_temp = df.loc[df['College'] == college].sample(n=sample_size, replace=False)
    small_df.append(df_temp)


df_samp = pd.concat(small_df, ignore_index=True)
df_samp

,Email,Department,College,TitleNew,CleanName,LastName
0,francisca.beer@ucr.edu,Other,University Extension,Lecturer,Francisca M Beer,Beer
1,paul.kirwan@ucr.edu,Other,University Extension,Lecturer,Paul Kirwan,Kirwan
2,nidhi.shah@ucr.edu,Other,University Extension,Lecturer,Nidhi Shah,Shah
3,lorna.seitz@ucr.edu,Other,University Extension,Lecturer,Lorna K Seitz,Seitz
4,michael.towne@ucr.edu,Other,University Extension,Lecturer,Michael R Towne,Towne
...,...,...,...,...,...,...
806,george.saffouri@ucr.edu,SOM Internal Medicine,School of Medicine,Assistant Professor,George Saffouri,Saffouri
807,maurizio.pellecchia@ucr.edu,Biomedical Sciences,School of Medicine,Professor,Maurizio Pellecchia,Pellecchia
808,samuel.wong@ucr.edu,SOM Internal Medicine,School of Medicine,Associate Professor,Samuel Wong,Wong
809,howard.hlee@ucr.edu,SOM Medicine,School of Medicine,Assistant Professor,Howard Lee,Lee


In [49]:
df_samp['Id'] = df_samp.index + 1
df_samp

,Email,Department,College,TitleNew,CleanName,LastName,Id
0,francisca.beer@ucr.edu,Other,University Extension,Lecturer,Francisca M Beer,Beer,1
1,paul.kirwan@ucr.edu,Other,University Extension,Lecturer,Paul Kirwan,Kirwan,2
2,nidhi.shah@ucr.edu,Other,University Extension,Lecturer,Nidhi Shah,Shah,3
3,lorna.seitz@ucr.edu,Other,University Extension,Lecturer,Lorna K Seitz,Seitz,4
4,michael.towne@ucr.edu,Other,University Extension,Lecturer,Michael R Towne,Towne,5
...,...,...,...,...,...,...,...
806,george.saffouri@ucr.edu,SOM Internal Medicine,School of Medicine,Assistant Professor,George Saffouri,Saffouri,807
807,maurizio.pellecchia@ucr.edu,Biomedical Sciences,School of Medicine,Professor,Maurizio Pellecchia,Pellecchia,808
808,samuel.wong@ucr.edu,SOM Internal Medicine,School of Medicine,Associate Professor,Samuel Wong,Wong,809
809,howard.hlee@ucr.edu,SOM Medicine,School of Medicine,Assistant Professor,Howard Lee,Lee,810


In [50]:
df_samp.to_csv('../data/sampleInstructorData.csv', index=False)

Created a spread sheet for each stratified population size, their proportions and each stratified sample size along with their proportions. The stratified sample sizes/proportions are subject to because its unlikely that I will get a response from everyone. Its also possible the information on the website is old and they are no longer part of that college. 

10. [X] Perform a non proportional stratified random sample of college/organization